In [ ]:
%matplotlib inline
from __future__ import division, print_function

import numpy as np
import matplotlib.pyplot as plt

# Solução da equação de flexura por diferenças finitas
Agora vamos resolver a equação de flexura 
$$D \frac{\partial^4 w}{\partial x^4} + \Delta \rho g w = p$$
numericamente através do método das diferenças finitas:
$$D 
\frac{w_{i+2} -4w_{i+1} +6w_{i} -4w_{i-1}+w_{i-2}}
{\Delta x^4}
+ \Delta \rho g w_i = p_i.$$

Rearranjando os termos temos:
$$D w_{i-2} - 4Dw_{i-1} 
+ \left[ 6D + \Delta x^4 \Delta \rho g \right] w_i
- 4Dw_{i-1} + D w_{i-2} = \Delta x^4 p_i   
$$

$$
\begin{bmatrix}
D & -4D & 6D+\Delta x^4 \Delta \rho g & -4D & D
\end{bmatrix}
\begin{bmatrix}
w_{i-2} \\ w_{i-1} \\ w_i \\ w_{i+1} \\ w_{i+2}
\end{bmatrix}=\Delta x^4 p_i
$$

Combinando as equações, forma-se um sistema linear. Por exemplo, para o caso de 5 pontos temos o seguinte sistema:

$$
\begin{bmatrix}
6D+\Delta x^4 \Delta \rho g & -4D & D & 0 & 0 \\
-4D & 6D+\Delta x^4 \Delta \rho g & -4D & D & 0 \\
D & -4D & 6D+\Delta x^4 \Delta \rho g & -4D & D \\
0 & D & -4D & 6D+\Delta x^4 \Delta \rho g & -4D \\
0 & 0 & D & -4D & 6D+\Delta x^4 \Delta \rho g \\
\end{bmatrix}
\begin{bmatrix}
w_0 \\ w_1 \\ w_2 \\ w_3 \\ w_4
\end{bmatrix}=
\Delta x^4
\begin{bmatrix}
p_0 \\ p_1 \\ p_2 \\ p_3 \\ p_4
\end{bmatrix}
$$

$$
\textbf{Aw}=\Delta x^4\textbf{p}
$$

O escalar $\Delta x^4$ poderia ser fundido ao vetor $\textbf{p}$, mas foi mantido em evidência por clareza.

# Construção do modelo numérico

Primeiramente vamos declarar os parâmetros do modelo e criar o vetor das posições $x$:

In [ ]:
drho = 2300.0
g = 10.0
E = 1.0E11
nu = 0.25
Te = 10000.0
V0 = -1.0E13

D = E*Te**3/(12*(1-nu**2))

L=1.0E6
dx = 5000.0
N = int(L/dx + 1)
x = np.linspace(-L/2,L/2,N) # np.linspace(x_inicial, x_final, número de pontos)

Agora vamos criar a matriz $\textbf A$ e o vetor $\textbf p$ usando o método __zeros()__

Para preencher a matriz $\textbf A$, podemos utilizar apenas 5 comandos, um para cada uma das diagonais não-nulas da matriz. Por exemplo, para a diagonal principal, todos os elementos irão receber o mesmo valor: $6D+\Delta x^4 \Delta \rho g$

$$
\begin{bmatrix}
6D+\Delta x^4 \Delta \rho g & . & . & . & . \\
. & 6D+\Delta x^4 \Delta \rho g & . & . & . \\
. & . & 6D+\Delta x^4 \Delta \rho g & . & . \\
. & . & . & 6D+\Delta x^4 \Delta \rho g & . \\
. & . & . & . & 6D+\Delta x^4 \Delta \rho g \\
\end{bmatrix}
$$

Com a função __range__ todos os elementos da diagonal poderão receber este valor com apenas um comando:

```Python
A[range(N),range(N)]= 6*D + dx**4*drho*g
```

Para a primeira diagonal acima da diagonal principal:
$$
\begin{bmatrix}
. & -4D & . & . & . \\
. & . & -4D & . & . \\
. & . & . & -4D & . \\
. & . & . & . & -4D \\
. & . & . & . & .   \\
\end{bmatrix}
$$ 
podemos usar um procedimento semelhante:
```Python
A[range(0,N-1),range(1,N)]=-4*D
```

Usando assim apenas 5 comandos (2 comandos já apresentados + 3 para as outras diagonais), construa a matriz $\textbf A$ 

Para uma carga concentrada na origem, como vimos para o caso da solução analítica, basta modificar apenas um elemento do vetor $\textbf p$ relativo ao ponto em $x=0$, representado pelo ponto central: `p[N/2]`. 

A unidade de $p$ é Pa = N/m$^2$ (força por unidade de área) enquanto que a unidade da carga concentrada $V_0$ é N/m (força por unidade de comprimento). Assim não podemos atribuir o valor de $V_0$ diretamente ao elemento `p[N/2]`.

Para haver consistência dimensional, o valor de $V_0$ deve ser dividido pelo espaçamento $\Delta x$ entre os pontos: `p[N//2]=V0/dx`.

Resta resolver o sistema linear $\textbf{Aw}=\Delta x^4\textbf{p}$ para obter o vetor $\textbf w$. Para isso use o método `linalg.solve` do `numpy`.

Vamos ver o resultado, plotando $w$ em função de $x$:

Para verificar se a solução numérica é consistente com a solução analítica, vamos copiar a função da solução analítica do exercício anterior para este notebook:

Vamos plotar a diferença entre o modelo analítico e o numérico em função de $x$ 